# Utils

### Create Run-scripts

In [ ]:
import os
import numpy as np
import math
if True:
    from tqdm import trange
    abstract_runscript_content = """#!/bin/tcsh
#
# This is a StarExec runscript for PyRes. See https://www.starexec.org.
#
# To use this, build a StarExec package with "make starexec" and
# upload it to a StarExec cluster - then follow local instructions
# ;-).
#
source /etc/profile.d/modules.csh
setenv MODULEPATH /starexec/sw/modulefiles/Linux:/starexec/sw/modulefiles/Core:/starexec/sw/lmod/lmod/modulefiles/Core
module load python/3.13.0
python --version
which python3.13.0
echo -n "% Problem    : " ; head -2 $1 | tail -1 | sed -e "s/.*  : //"
set ProblemSPC=`grep " SPC " $1 | sed -e "s/.* : //"`
set default_flags="-tifbsVp -nlargest -HPickGiven5"
set rel_flags="-r [rel_distance]"
set final=" "$1
set ecmd="python3 ./pyres_fof.py $default_flags $rel_flags $final"
python3 --version
if ( `expr "$ProblemSPC" : "FOF.*"` || `expr "$ProblemSPC" : "CNF.*"` ) then
	echo "% Command    : " $ecmd
	/home/starexec/bin/GetComputerInfo -p THIS Model CPUModel RAMPerCPU OS | \
        sed -e "s/Computer     /% Computer   /" \
            -e "s/Model        /% Model      /" \
            -e "s/CPUModel     /% CPU        /" \
            -e "s/RAMPerCPU    /% Memory     /" \
            -e "s/OS           /% OS         /"
	echo -n "% CPULimit   : " ; echo "$STAREXEC_CPU_LIMIT"
	echo -n "% DateTime   : " ; date
	echo "% CPUTime    : "
	$ecmd
else
    echo "% SZS status Inappropriate"
endif
    """

    abstract_file_name = "run_scripts/starexec_run_PyRes_rd_[rel_distance]"

    for f in os.listdir("run_scripts/"):
        os.remove(os.path.join("run_scripts/", f))

    X = np.arange(0, 10)
    def get_rel_distances(X, end_of_linear):
        a = 10
        steepness_expo = 1.5
        additive = -(end_of_linear/a)+steepness_expo
        return np.real(np.where(
            X<=(end_of_linear/a),
            a*X,
            a*(np.emath.logn(steepness_expo, X+additive)*(X+additive)-additive)
        )).astype(int).tolist()

    rel_distances = get_rel_distances(X, 50)
    rel_distances = [2, 3, 5, 8, 13, 21, 33, 54]
    print(rel_distances)

    for x in tqdm(rel_distances):
        def substitute(text): return text.replace("[rel_distance]", f"{x:03d}")

        file_name = substitute(abstract_file_name)
        file_content = substitute(abstract_runscript_content)
        with open(file_name, "w") as file:
            file.write(file_content)

[2, 3, 5, 8, 13, 21, 33, 54]


100%|██████████| 8/8 [00:00<00:00, 677.98it/s]
